In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [2]:
from scipy.interpolate import interp1d
# resample target
def resample_cycle(cycle, num_points):
    """Resamples a gait cycle to have a fixed number of points using interpolation."""
    original_indices = np.linspace(0, 1, len(cycle))  # original indices based on the length of the cycle
    new_indices = np.linspace(0, 1, num_points)  # new indices to match the desired number of points
    interpolation_function = interp1d(original_indices, cycle, kind='linear')  # interpolate the data
    return interpolation_function(new_indices)  # return the resampled cycle

## Stick insect pattern

In [3]:
leg_0_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_0.csv')
leg_1_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_1_v1.csv')
leg_2_df = pd.read_csv('include/stick_insect_path/rbfn_joint_targets_2_new_v1.csv')

fig = go.Figure()
fig = make_subplots(rows=3, cols=3, shared_xaxes=False)
fig.update_layout(height=800, width=1100, title="Path plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=leg_0_df['TR0'], mode='lines', name=f'TR0'), row=1, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['CR0'], mode='lines', name=f'CR0'), row=2, col=1)
fig.add_trace(go.Scatter( y=leg_0_df['FR0'], mode='lines', name=f'FR0'), row=3, col=1)

fig.add_trace(go.Scatter( y=leg_1_df['TR1'], mode='lines', name=f'TR1'), row=1, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['CR1'], mode='lines', name=f'CR1'), row=2, col=2)
fig.add_trace(go.Scatter( y=leg_1_df['FR1'], mode='lines', name=f'FR1'), row=3, col=2)

fig.add_trace(go.Scatter( y=leg_2_df['TR2'], mode='lines', name=f'TR2'), row=1, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['CR2'], mode='lines', name=f'CR2'), row=2, col=3)
fig.add_trace(go.Scatter( y=leg_2_df['FR2'], mode='lines', name=f'FR2'), row=3, col=3)

## CPG(SO2)-RBF

In [4]:
from include.cpg_rbf.cpg_so2 import CPG_SO2
# from include.cpg_rbf.cpg_afdc import CPG_AFDC
from include.cpg_rbf.rbf import RBF

rbf = RBF()
cpg = CPG_SO2()
cpg_one_cycle = cpg.generate_cpg_one_cycle(0.05)    # lowest phi value
out0_cpg_one_cycle = cpg_one_cycle['out0_cpg_one_cycle'][:]
out1_cpg_one_cycle = cpg_one_cycle['out1_cpg_one_cycle'][:]
cpg_cycle_length = len(out0_cpg_one_cycle)
cpg_kernel = rbf.construct_kernels_with_cpg_one_cycle(out0_cpg_one_cycle, out1_cpg_one_cycle, cpg_cycle_length) # construct kernels with cpg one cycle

In [5]:
columns = ['TR0', 'CR0', 'FR0']
imitated_paths = {}
imitated_weights = {}

for col in columns:
    # Resample cycle for the current column
    target = resample_cycle(np.array(leg_0_df[col]), cpg_cycle_length)
    
    # Imitate path and get weights using RBF
    imitated_paths[col]     = rbf.imitate_path_by_learning(target)
    imitated_weights[col]   = rbf.get_rbf_weight()

# print(imitated_weight_1)
# print(len(imitated_weight_1))
# print(rbf.error_max)

last iteration: 11
last iteration: 9
last iteration: 17


In [6]:
np.savez('imitated_weights.npz', **imitated_weights)

# loaded_weights = np.load('imitated_weights.npz')
# for key in loaded_weights:
#     print(key, loaded_weights[key])

In [7]:

fig = go.Figure()
fig = make_subplots(rows=5, cols=1, shared_xaxes=False)
fig.update_layout(height=800, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

for i in range(len(cpg_kernel)):
    fig.add_trace(go.Scatter( y=cpg_kernel[i], mode='lines', name=f'kernels{i}'), row=2, col=1)

fig.add_trace(go.Scatter( y=out0_cpg_one_cycle, mode='lines', name=f'out0_cpg_one_cycle'), row=1, col=1)
fig.add_trace(go.Scatter( y=out1_cpg_one_cycle, mode='lines', name=f'out1_cpg_one_cycle'), row=1, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['TR0']), cpg_cycle_length), mode='lines', name=f'TR0'), row=3, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=4, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['CR0']), cpg_cycle_length), mode='lines', name=f'CR0'), row=4, col=1)

fig.add_trace(go.Scatter( y=imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=5, col=1)
fig.add_trace(go.Scatter( y=resample_cycle(np.array(leg_0_df['FR0']), cpg_cycle_length), mode='lines', name=f'FR0'), row=5, col=1)

In [8]:
cpg = CPG_SO2()
cpg_output_0 = []
cpg_output_1 = []
online_imitated_paths = {'TR0':[], 'CR0':[], 'FR0':[]}
for i in range(4500):
    cpg.s = 1.0
    cpg.update_cpg(0.05)      # phi would be equal when learning


    cpg_output_0.append(cpg.out0_t)
    cpg_output_1.append(cpg.out1_t)
    
    online_imitated_paths['TR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR0']))
    online_imitated_paths['CR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR0']))
    online_imitated_paths['FR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR0']))

fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.update_layout(height=600, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)


## Pause-Resume by adjust parameters of CPG

In [ ]:
cpg = CPG_SO2()
cpg_output_0 = []
cpg_output_1 = []
online_imitated_paths = {'TR0':[], 'CR0':[], 'FR0':[]}
for i in range(4500):
    
    if i < 1500:
        cpg.output_cpg_weight = 1.0
        cpg.update_cpg(0.05)      # phi would be equal when learning
    elif i > 1500 and i < 3000:
        cpg.output_cpg_weight = 1000.0  
        cpg.w00 = 0.001
        cpg.w01 = 0 
        cpg.w10 = 0 
        cpg.w11 = 0.001 
        cpg.update_cpg(None)
    elif i > 3000:
        cpg.output_cpg_weight = 1.0
        cpg.update_cpg(0.05)      # phi would be equal when learning

    cpg_output_0.append(cpg.out0_t)
    cpg_output_1.append(cpg.out1_t)
    
    online_imitated_paths['TR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR0']))
    online_imitated_paths['CR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR0']))
    online_imitated_paths['FR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR0']))

fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.update_layout(height=600, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)


## Rewind

In [10]:
cpg = CPG_SO2()
cpg_output_0 = []
cpg_output_1 = []
online_imitated_paths = {'TR0':[], 'CR0':[], 'FR0':[]}
    
random_number = np.random.randint(0, 4501)
for i in range(4500):
    if i < random_number:
        cpg.output_cpg_weight = 1.0
        cpg.w00 = cpg.alpha * np.cos(0.05)
        cpg.w01 = cpg.alpha * np.sin(0.05)
        cpg.w10 = cpg.alpha * (-np.sin(0.05))
        cpg.w11 = cpg.alpha * np.cos(0.05)
        cpg.update_cpg(None)      # phi would be equal when learning
        cpg_output_0.append(cpg.out0_t)
        cpg_output_1.append(cpg.out1_t)
    elif i >= random_number:
        cpg.output_cpg_weight = 1.0
        # cpg.w00 = cpg.alpha * np.cos(-0.05)
        # cpg.w01 = cpg.alpha * np.sin(-0.05)
        # cpg.w10 = cpg.alpha * (-np.sin(-0.05))
        # cpg.w11 = cpg.alpha * np.cos(-0.05)
        cpg.update_cpg(-0.05)
        cpg_output_0.append(cpg.out0_t)
        cpg_output_1.append(cpg.out1_t)

    
    online_imitated_paths['TR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['TR0']))
    online_imitated_paths['CR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['CR0']))
    online_imitated_paths['FR0'].append(rbf.regenerate_target_traj(cpg.out0_t, cpg.out1_t, imitated_weights['FR0']))

fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.update_layout(height=600, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)


## CPG-RBF modulation

In [11]:
from include.cpg_rbf.cpg_so2 import CPG_LOCO
cpg_loco = CPG_LOCO()
cpg_output_0 = []
cpg_output_1 = []
online_imitated_paths = {'TR0':[], 'CR0':[], 'FR0':[]}

for i in range(4500):
    if i < 1000:
        cpg_out = cpg_loco.modulate_cpg(0.05, 0.0, 0.0)
        
        cpg_output_0.append(cpg_out['cpg_output_0'])
        cpg_output_1.append(cpg_out['cpg_output_1'])
    elif i >= 1000 :
        cpg_out = cpg_loco.modulate_cpg(0.05, 0.0, 1.0)
        
        cpg_output_0.append(cpg_out['cpg_output_0'])
        cpg_output_1.append(cpg_out['cpg_output_1'])
    # elif i >= 2000 and i < 3000:
    #     cpg_out = cpg_loco.modulate_cpg(0.05, 0.0, 0.0)
        
    #     cpg_output_0.append(cpg_out['cpg_output_0'])
    #     cpg_output_1.append(cpg_out['cpg_output_1'])
    # elif i >= 3000 and i < 4500:
    #     cpg_out = cpg_loco.modulate_cpg(0.05, 0.0, 1.0)
        
    #     cpg_output_0.append(cpg_out['cpg_output_0'])
    #     cpg_output_1.append(cpg_out['cpg_output_1'])
    
    online_imitated_paths['TR0'].append(rbf.regenerate_target_traj(cpg_out['cpg_output_0'], cpg_out['cpg_output_1'], imitated_weights['TR0']))
    online_imitated_paths['CR0'].append(rbf.regenerate_target_traj(cpg_out['cpg_output_0'], cpg_out['cpg_output_1'], imitated_weights['CR0']))
    online_imitated_paths['FR0'].append(rbf.regenerate_target_traj(cpg_out['cpg_output_0'], cpg_out['cpg_output_1'], imitated_weights['FR0']))

fig = go.Figure()
fig = make_subplots(rows=4, cols=1, shared_xaxes=True)
fig.update_layout(height=600, width=1100, title="CPG plot")
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))  # Reduce margins

fig.add_trace(go.Scatter( y=cpg_output_0, mode='lines', name=f'cpg_output_0'), row=1, col=1)
fig.add_trace(go.Scatter( y=cpg_output_1, mode='lines', name=f'cpg_output_1'), row=1, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['TR0'], mode='lines', name=f'TR0 (CPG-RBF)'), row=2, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['CR0'], mode='lines', name=f'CR0 (CPG-RBF)'), row=3, col=1)
fig.add_trace(go.Scatter( y=online_imitated_paths['FR0'], mode='lines', name=f'FR0 (CPG-RBF)'), row=4, col=1)




rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind
rewind

In [22]:
print(np.cos(np.pi/2.1))
print(np.cos(1.56979633))

0.07473009358642439
0.0009999966282316015
